<a href="https://www.kaggle.com/code/princeiornongu/tweet-nlp?scriptVersionId=142123930" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [2]:
df = pd.read_csv('/kaggle/input/cyberbullying-tweets/CyberBullying Comments Dataset.csv')
print(df.head(10))
print(df['Text'].apply(lambda x: len(x.split(' '))).sum())

                                                Text  CB_Label
0  damn there is someones nana up here at beach w...         0
1  no kidding! dick clark was a corpse mechanical...         0
2  i read an article on jobros and thought damn w...         0
3  I got one fucking day of sprinkles and now it'...         0
4  I was already listening to Elliott smith  and ...         0
5  I told Derek to go fuck himself  Devyn told us...         0
6  I'm watching the new Smosh video and laughing ...         0
7  My mom didn't like Catholicism because the ide...         0
8  ya i know! lol AND big thunder mountain broke ...         0
9                                  lucky bitch....."         0
191019


In [3]:
import re # simiply the review, not the library to stem the reviews 
import nltk # download ensemble of stop words, helps remove not revelant words to the reviews or machine 
nltk.download('stopwords') # gets all the stopwords, use later to not include the non relevant words  
from nltk.corpus import stopwords # imprt to notebook
from nltk.stem.porter import PorterStemmer # taking the root of the word that indicates what that words mean, breaking to its base form 
corpus = [] # create a new list, contain s diff review all clean and into a list, create a for loop with a cleaning process. only get cleaned reviews 
for i in range(0,1000):
    review = re.sub('[^a-zA-Z]', ' ', df['Text'][i]) # .sub function can replace anything in a text with anything you want anything in [] all lowercae as wel as the caps ^ = exactly not  
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
print(corpus)

['damn someon nana beach one dont think ic steal get quickli tho', 'kid dick clark corps mechan oper advertis compani', 'read articl jobro thought damn cash jobro pokemon perfect stock stuffer haha', 'got one fuck day sprinkl back sunshin douchebaggeri', 'alreadi listen elliott smith fuck hate kany west not valid artist human eye', 'told derek go fuck devyn told us calm', 'watch new smosh video laugh ass', 'mom like catholic idea sinner birth irk love gay peopl yeah', 'ya know lol big thunder mountain broke got ride got ton bad luck think u stole bitch', 'lucki bitch', 'hate auto dm better way show actual care follow put autopilot', 'top notch click dont hate hater u mad cuz u broke use playstat cd player', 'question fuck father mia like two day', 'fair enough get ass around sometim later voic chang btw p', 'yeah seem like got bitch somewher could brainwash got forum right that go bey', 'see friend mine use acai berri medic purpos shrug damn hella skinni may tri mayb', 'hate hugh lauri

In [5]:
from sklearn.model_selection import train_test_split 
X = df.Text
y = df.CB_Label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline


# Create a text preprocessing pipeline
text_preprocessing = Pipeline([
    ('count_vectorizer', CountVectorizer()),  # Convert text to word frequency vectors
    ('tfidf_transformer', TfidfTransformer()),  # Apply TF-IDF transformation
])

# Preprocess the training and testing data
X_train_transformed = text_preprocessing.fit_transform(X_train)
X_test_transformed = text_preprocessing.transform(X_test)

# Initialize and train CatBoostClassifier
catboost_classifier = CatBoostClassifier(iterations=1500, 
                                         learning_rate=0.1, 
                                         depth=7, 
                                         )  # CatBoost classifier
catboost_classifier.fit(X_train_transformed, y_train)

# Predict on the testing data
y_pred = catboost_classifier.predict(X_test_transformed)

# Print classification report
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
cm =confusion_matrix(y_test, y_pred)
print(cm)

acc=accuracy_score(y_test, y_pred)
print(acc)
print(f" Model Accuracy: {acc * 100:.2f}%")


0:	learn: 0.6785029	total: 247ms	remaining: 6m 9s
1:	learn: 0.6627973	total: 364ms	remaining: 4m 32s
2:	learn: 0.6520142	total: 484ms	remaining: 4m 1s
3:	learn: 0.6413503	total: 603ms	remaining: 3m 45s
4:	learn: 0.6346016	total: 720ms	remaining: 3m 35s
5:	learn: 0.6267829	total: 838ms	remaining: 3m 28s
6:	learn: 0.6193652	total: 955ms	remaining: 3m 23s
7:	learn: 0.6130861	total: 1.07s	remaining: 3m 19s
8:	learn: 0.6098341	total: 1.19s	remaining: 3m 16s
9:	learn: 0.6062201	total: 1.3s	remaining: 3m 14s
10:	learn: 0.6033356	total: 1.42s	remaining: 3m 12s
11:	learn: 0.5995039	total: 1.57s	remaining: 3m 14s
12:	learn: 0.5956826	total: 1.71s	remaining: 3m 15s
13:	learn: 0.5927402	total: 1.83s	remaining: 3m 13s
14:	learn: 0.5898965	total: 1.97s	remaining: 3m 15s
15:	learn: 0.5873797	total: 2.09s	remaining: 3m 13s
16:	learn: 0.5852344	total: 2.2s	remaining: 3m 12s
17:	learn: 0.5826347	total: 2.33s	remaining: 3m 11s
18:	learn: 0.5806073	total: 2.46s	remaining: 3m 11s
19:	learn: 0.5785525	total

In [7]:
#76.35% only at catboost (1500, 0.1, 7)

In [8]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = catboost_classifier, X = X_train_transformed, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

0:	learn: 0.6797497	total: 149ms	remaining: 3m 43s
1:	learn: 0.6683396	total: 260ms	remaining: 3m 14s
2:	learn: 0.6581667	total: 373ms	remaining: 3m 6s
3:	learn: 0.6497239	total: 484ms	remaining: 3m 1s
4:	learn: 0.6417535	total: 596ms	remaining: 2m 58s
5:	learn: 0.6308302	total: 706ms	remaining: 2m 55s
6:	learn: 0.6225092	total: 817ms	remaining: 2m 54s
7:	learn: 0.6173471	total: 929ms	remaining: 2m 53s
8:	learn: 0.6118210	total: 1.04s	remaining: 2m 52s
9:	learn: 0.6076132	total: 1.15s	remaining: 2m 52s
10:	learn: 0.6041458	total: 1.26s	remaining: 2m 51s
11:	learn: 0.6014393	total: 1.37s	remaining: 2m 50s
12:	learn: 0.5978577	total: 1.48s	remaining: 2m 49s
13:	learn: 0.5941818	total: 1.59s	remaining: 2m 48s
14:	learn: 0.5916896	total: 1.7s	remaining: 2m 48s
15:	learn: 0.5899730	total: 1.8s	remaining: 2m 47s
16:	learn: 0.5871206	total: 1.91s	remaining: 2m 47s
17:	learn: 0.5851533	total: 2.02s	remaining: 2m 46s
18:	learn: 0.5822643	total: 2.13s	remaining: 2m 46s
19:	learn: 0.5804170	total